<a href="https://colab.research.google.com/github/dannynacker/octAVEs/blob/main/isomod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub

import pandas as pd
import numpy as np
# Workaround for librosa’s deprecated np.complex use
np.complex = complex

import librosa
from scipy.interpolate import interp1d
import soundfile as sf
from pydub import AudioSegment
import tempfile
import os

# ─────────────────────────────────────────────────────
#  Paths & settings
# ─────────────────────────────────────────────────────
csv_path       = "/content/Ashes_grouped_FFT.csv"  # your FFT CSV
audio_path     = "/content/Ashes.mp3"             # source audio (WAV/MP3/etc)
output_mp3     = "/content/Ashes_mod.mp3"         # desired MP3 filename

freq_prefix    = "Adjusted_Corr_Freq"          # FFT frequency columns
duty_prefix    = "Amplitude_RX1"               # FFT duty (%) columns

# ─────────────────────────────────────────────────────
#  Load data
# ─────────────────────────────────────────────────────
df = pd.read_csv(csv_path)
y, sr = librosa.load(audio_path, sr=44100, mono=True)

t_csv   = df["Time"].values
t_audio = np.arange(len(y)) / sr
channels = [1, 2, 3, 4]

# ─────────────────────────────────────────────────────
#  Interpolate & modulate
# ─────────────────────────────────────────────────────
# Build interpolation functions
freqs, duties = {}, {}
for ch in channels:
    freqs[ch]  = interp1d(t_csv, df[f"{freq_prefix}_{ch}"], kind="linear", fill_value="extrapolate")(t_audio)
    duties[ch] = np.clip(interp1d(t_csv, df[f"{duty_prefix}_{ch}"] / 100.0,
                         kind="linear", fill_value="extrapolate")(t_audio), 0, 1)

# Generate 4 square‐wave envelopes and apply them
env = np.zeros((4, len(y)))
for ch in channels:
    # cumulative phase
    phase = 2 * np.pi * np.cumsum(freqs[ch]) / sr
    # cycle position in [0,1)
    cycle_pos = (phase / (2 * np.pi)) % 1.0
    # envelope = 1 when within duty fraction
    env[ch-1] = (cycle_pos < duties[ch]).astype(float)

# Mix the four modulated tracks
y_mod = sum(env_i * y for env_i in env)

# Normalize to 90% of full scale
peak = np.max(np.abs(y_mod))
if peak > 0:
    y_mod = y_mod / peak * 0.9

# ─────────────────────────────────────────────────────
#  Export to MP3
# ─────────────────────────────────────────────────────
# Write to temporary WAV, then convert to MP3
tmpfile = tempfile.mktemp(suffix=".wav")
sf.write(tmpfile, y_mod, sr)
audio = AudioSegment.from_wav(tmpfile)
audio.export(output_mp3, format="mp3", bitrate="192k")
os.remove(tmpfile)

print(f"Modulated MP3 written to: {output_mp3}")